In [96]:
from IPython.display import display
import pickle
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

In [97]:
import cafle as cf
from cafle.genfunc import rounding as R
from cafle.genfunc import PY
from cafle.genfunc import EmptyClass

In [98]:
pd.options.display.max_columns = 40
pd.options.display.max_rows = 200
print(pd.get_option("display.max_columns"), 
      pd.get_option("display.max_rows"))

40 200


# Introduction

* Logistic center development

# Initial Setting

In [99]:
astn = EmptyClass()

# Input Index Data

In [100]:
tmp_col = ['idxname', 'start', 'periods']
tmp_ary = [['prjt',  '2021-12', 35],
           ['cstrn', '2022-01', 23],
           ['loan',  '2022-01', 29],
           ['sales', '2022-01', 29]]
astn.index = DataFrame(tmp_ary, columns=tmp_col)

idx = cf.PrjtIndex(idxname = list(astn.index.idxname),
                   start   = list(astn.index.start),
                   periods = list(astn.index.periods + 1),
                   freq    = 'M')

# Input Financing Condition Data

In [101]:
tmp_col = ['title', 'amt_ntnl', 'amt_intl']
tmp_ary = [['equity',  1_300,     1_300]]
astn.equity = DataFrame(tmp_ary, columns=tmp_col)

equity = cf.Loan(idx,
                 amt_ntnl = astn.equity.amt_ntnl[0],
                 amt_intl = astn.equity.amt_intl[0])

In [102]:
tmp_col = ['title', 'amt_ntnl', 'amt_intl', 'rate_fee', 'rate_IR']
tmp_ary = [['tra',    75_000,     18_000,      0.020,     0.043],
           ['trb',    22_000,     22_000,      0.041,     0.070]]
astn.loan = DataFrame(tmp_ary, columns=tmp_col)

loan = cf.Intlz_loan(idx, idx.loan,
                     title = astn.loan.title,
                     amt_ntnl = astn.loan.amt_ntnl,
                     amt_intl = astn.loan.amt_intl,
                     rate_fee = astn.loan.rate_fee,
                     rate_IR = astn.loan.rate_IR)

# 미인출잔액수수료율
loan.feeonbal_tra = 0.005

print("## Financing Condition ##\n")
print(f"Total Equity amount:{sum(astn.equity.amt_ntnl):>7,.0f}")
print(f"Total loan amount:  {sum(astn.loan.amt_ntnl):>7,.0f}")
print(R(astn.loan, rate=['rate_fee', 'rate_IR']))

## Financing Condition ##

Total Equity amount:  1,300
Total loan amount:   97,000
  title amt_ntnl amt_intl rate_fee rate_IR
0   tra   75,000   18,000     2.00    4.30
1   trb   22,000   22,000     4.10    7.00


# Input Sales Data

In [103]:
# Pickling Data
with open("data/valuation.pickle", "rb") as fr:
    data_valuation = pickle.load(fr)
valuation = data_valuation['valuation_pivot']
value = valuation.loc[[('23,000', '50,000')]].values[0]
print("## Valuation Matrix ##")
display(R(valuation))

## Valuation Matrix ##


cap_rate,"(4.0, 4.6)","(4.1, 4.8)","(4.2, 5.0)","(4.3, 5.2)","(4.4, 5.4)"
rent,,,,,
"(21,000, 46,000)","123,728","119,432","115,441","111,721","108,246"
"(22,000, 48,000)","128,976","124,499","120,339","116,463","112,840"
"(23,000, 50,000)","134,224","129,566","125,238","121,204","117,435"
"(24,000, 52,000)","139,472","134,633","130,136","125,945","122,029"
"(25,000, 54,000)","144,720","139,700","135,034","130,687","126,624"
"(26,000, 56,000)","149,969","144,766","139,933","135,428","131,218"


In [104]:
sales_value = 130_000
sales = cf.Account(idx, 'Sales')
sales.addamt(idx.sales[0], sales_value)
sales.subscdd(idx.sales[-1], sales_value)

# Input Cost Data

### 1) Set Cost Class

In [105]:
cost = cf.Cost(idx)

### 2) Land Purchase Costs

In [106]:
title = ('lnd', 'prchs') # land purchase cost
byname = '용지매입비'
tmp_ary = [[idx.locval(2021, 12),  1_000],
           [idx.locval(2022,  1), 13_500]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [107]:
title = ('lnd', 'brkrg') # land brokerage fee
byname = '부동산중개비용'
tmp_ary = [[idx.locval(2021, 12),  39],
           [idx.locval(2022,  1),  91]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [108]:
title = ('lnd', 'tax_aqstn') # land acquisition and registration tax
byname = '취등록세'
tmp_ary = [[idx.locval(2022, 1), 675]]
# 토지매매대금 x 취득세 4.0%, 농특세 0.2%, 교육세 0.4%
cost.inptcst(title, byname, scdddf=tmp_ary)

In [109]:
title = ('lnd', 'nhbond') # national housing bond
byname = '국민주택채권'
tmp_ary = [[idx.locval(2022, 1), 36]]
# 토지매매대금 x 매입률 4.5% x 본인부담율 5.5%
cost.inptcst(title, byname, scdddf=tmp_ary)

In [110]:
title = ('lnd', 'jdclcst') # judicial scrivener cost
byname = '법무사비용'
tmp_ary = [[idx.locval(2022, 1), 15]]
# 토지매매대금 x 0.1%
cost.inptcst(title, byname, scdddf=tmp_ary)

### 3) Construction Costs

In [111]:
cstrn = {}
cstrn['dmltn'] = 35 + 1_180 # 철거비: 3,550평 x 342천원, 온천공 폐공공사 포함
cstrn['civil'] = 8_490 # 토목공사: 16,327평 x 520천원
cstrn['nml_fclt'] = 9_311 * 2.66 # 상온시설: 9,311평 x 2,660천원
cstrn['cld_fclt'] = 8_365 * 3.66 # 저온시설: 8,365평 x 3,660천원
cstrn_sum = round(sum([cstrn[key] for key in cstrn]))

retention_rate = 0.1
cstrn_intl = 35
cstrn_prd = cstrn_sum * (1-retention_rate) - cstrn_intl
cstrn_rttn = cstrn_sum * retention_rate

print(f"Total construction cost:   {cstrn_sum:>7,.0f}")
print(f"Initial construction cost: {cstrn_intl:>7,.0f}")
print(f"Period construction cost:  {cstrn_prd:>7,.0f}")
print(f"Retained construction cost:{cstrn_rttn:>7,.0f}")

Total construction cost:    65,088
Initial construction cost:      35
Period construction cost:   58,544
Retained construction cost:  6,509


In [112]:
title = ('cstrn', 'cstrn') # construction cost
byname = '공사비'
tmp_idx = idx.cstrn.index
tmp_len = len(idx.cstrn)
tmp_ary = [[idx.prjt[0], cstrn_intl]]
tmp_ary.extend([[x, cstrn_prd / tmp_len] for x in tmp_idx])
tmp_ary.append([idx.prjt[-1], cstrn_rttn])

cost.inptcst(title, byname, scdddf=tmp_ary)

### 4) Indirect Construction Costs

In [113]:
title = ('cstrnidrt', 'wtrelec') # water supply, electricity etc. 
byname = '각종인입비'
tmp_ary = [[idx.locval(2023, 10), 980]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [114]:
title = ('cstrnidrt', 'wsctbn') # water and sewage contribution
byname = '상수도분담금'
tmp_ary = [[idx.locval(2023, 10), 163]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [115]:
title = ('cstrnidrt', 'dsgncst') # design cost
byname = '설계비'
tmp_ary = [[idx.locval(2022, 1), 624]]
# 16,327평 x 38천원
cost.inptcst(title, byname, scdddf=tmp_ary)

In [116]:
title = ('cstrnidrt', 'lcnscst') # license cost
byname = '인허가비용'
tmp_ary = [[idx.locval(2022, 1), 528]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [117]:
title = ('cstrnidrt', 'spvsn') # supervision cost
byname = '감리비'
ttl_amt = 490 # 16,327평 x 30천원, 월별 분할 부과
tmp_idx = idx.cstrn.index
tmp_len = len(idx.cstrn)
tmp_ary = [[x, ttl_amt / tmp_len] for x in tmp_idx]
cost.inptcst(title, byname, scdddf=tmp_ary)

### 5) Marketing Costs

In [118]:
title = ('mrktg', 'rentagncy') # rent agency cost
byname = '임대대행수수료'
tmp_ary = [[idx.prjt[-1], 1_070]]
# 임대계약 실행시 월 임대료의 2개월치 지급
# 1개월치 임대료 517백만원 가정
# 전액 성공불 지급
cost.inptcst(title, byname, scdddf=tmp_ary)

In [119]:
title = ('mrktg', 'salescsltg') # sales consulting fee
byname = '매각컨설팅수수료'
fee_amt = 130_000 * 0.008 # 매각가 x 0.8%
tmp_ary = [[idx.loan[0], fee_amt]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [120]:
title = ('mrktg', 'advtsmnt') # advertisement and promotion cost
byname = '광고홍보비'
tmp_ary = [[idx.locval(2022, 6), 250],
           [idx.locval(2023, 6), 250]]
cost.inptcst(title, byname, scdddf=tmp_ary)

### 6) Tax and Utility bills(제세공과금)

In [121]:
title = ('taxutlt', 'prpttax') # property tax
byname = '재산세종부세'
tmp_ary = [[idx.locval(2022, 6), 37],
           [idx.locval(2023, 6), 37]]
# 5,590 x 0.62% 2년
cost.inptcst(title, byname, scdddf=tmp_ary)

In [122]:
title = ('taxutlt', 'prsvtntax') # preservation registration fee
byname = '보존등기비'
tmp_ary = [[idx.cstrn.index[-1], 2_866]]
# 건물 취득비용 x 3.40%(취득세 2.8%, 농특세 0.2%, 교육세 0.16%, 법무사 0.24%)
cost.inptcst(title, byname, scdddf=tmp_ary)

### 7) Additional Costs

In [123]:
title = ('addtnl', 'pmfee') # Additional expense, PM fee
byname = 'PM수수료'
tmp_ary = [[idx.locval(2022, 1), 200]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [124]:
title = ('addtnl', 'oprtgcst') # company operating cost
unt_amt = 30 # 3,000만원/월
tmp_idx = idx.loan.index
tmp_ary = [[x, unt_amt] for x in tmp_idx]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [125]:
title = ('addtnl', 'rsrvfund') # reserve fund
byname = '예비비'
tmp_ary = [[idx.locval(2022, 6), 940],
           [idx.locval(2023, 6), 939]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [126]:
title = ('addtnl', 'trustcst') # trust fee
byname = '신탁수수료'
tmp_ary = [[idx.locval(2022, 1), 1_879]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [127]:
title = ('addtnl', 'csltcst') # consulting fee
byname = '사평감평용역비'
tmp_ary = [[idx.locval(2021, 12), 10],
           [idx.locval(2022, 1), 140]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [128]:
title = ('addtnl', 'legalcst') # legal advice fee
byname = '법률자문'
tmp_ary = [[idx.locval(2022, 1), 40]]
cost.inptcst(title, byname, scdddf=tmp_ary)

### 8) Financing Costs

In [129]:
title = ('fncg', 'arngmnt') # financing arrangement fee
byname = '금융주관수수료'
tmp_rate = 0.02 #2.0%
tmp_amt = sum(loan.amt_ntnl)
tmp_ary = [[idx.locval(2022, 1), tmp_amt * tmp_rate]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [130]:
title = ('fncg', 'spcoprtg') # spc operating cost
byname = 'SPC유동화비용'
tmp_ary = [[idx.locval(2022, 1), 66]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [131]:
title = ('fncg', 'agntbank') # agent banking fee
byname = '대리금융기관'
tmp_ary = [[idx.locval(2022, 1), 30],
           [idx.locval(2023, 1), 30]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [132]:
title = ('fncg', 'prepchs_fee') # pre-purchase agreement fee
byname = '선매입약정수수료'
tmp_ary = [[idx.locval(2022, 1), 0]]
cost.inptcst(title, byname, scdddf=tmp_ary)

### 9) Cost Matrix

In [133]:
cost_dct = cost["all"].dct.items()
cost_df = DataFrame({key:item.add_scdd[:] for key, item in cost_dct})
cost_df[("sum", "sum")] = cost["all"].add_scdd[:]
display(R(cost_df))

lnd                                  cstrn cstrnidrt         \
             prchs  brkrg tax_aqstn nhbond jdclcst  cstrn   wtrelec wsctbn   
2021-12-31   1,000  39.00         0   0.00    0.00     35         0      0   
2022-01-31  13,500  91.00       675  36.00   15.00  2,439         0      0   
2022-02-28       0   0.00         0   0.00    0.00  2,439         0      0   
2022-03-31       0   0.00         0   0.00    0.00  2,439         0      0   
2022-04-30       0   0.00         0   0.00    0.00  2,439         0      0   
2022-05-31       0   0.00         0   0.00    0.00  2,439         0      0   
2022-06-30       0   0.00         0   0.00    0.00  2,439         0      0   
2022-07-31       0   0.00         0   0.00    0.00  2,439         0      0   
2022-08-31       0   0.00         0   0.00    0.00  2,439         0      0   
2022-09-30       0   0.00         0   0.00    0.00  2,439         0      0   
2022-10-31       0   0.00         0   0.00    0.00  2,439         0      0   
2022-11-30       0   0.00         0   0.00    0.00  2,439         0      0   
2022-12-31       0   0.00         0   0.00    0.00  2,439         0      0   
2023-01-31       0   0.00         0   0.00    0.00  2,439         0      0   
2023-02-28       0   0.00         0   0.00    0.00  2,439         0      0   
2023-03-31       0   0.00         0   0.00    0.00  2,439         0      0   
2023-04-30       0   0.00         0   0.00    0.00  2,439         0      0   
2023-05-31       0   0.00         0   0.00    0.00  2,439         0      0   
2023-06-30       0   0.00         0   0.00    0.00  2,439         0      0   
2023-07-31       0   0.00         0   0.00    0.00  2,439         0      0   
2023-08-31       0   0.00         0   0.00    0.00  2,439         0      0   
2023-09-30       0   0.00         0   0.00    0.00  2,439         0      0   
2023-10-31       0   0.00         0   0.00    0.00  2,439       980    163   
2023-11-30       0   0.00         0   0.00    0.00  2,439         0      0   
2023-12-31       0   0.00         0   0.00    0.00  2,439         0      0   
2024-01-31       0   0.00         0   0.00    0.00      0         0      0   
2024-02-29       0   0.00         0   0.00    0.00      0         0      0   
2024-03-31       0   0.00         0   0.00    0.00      0         0      0   
2024-04-30       0   0.00         0   0.00    0.00      0         0      0   
2024-05-31       0   0.00         0   0.00    0.00      0         0      0   
2024-06-30       0   0.00         0   0.00    0.00      0         0      0   
2024-07-31       0   0.00         0   0.00    0.00      0         0      0   
2024-08-31       0   0.00         0   0.00    0.00      0         0      0   
2024-09-30       0   0.00         0   0.00    0.00      0         0      0   
2024-10-31       0   0.00         0   0.00    0.00      0         0      0   
2024-11-30       0   0.00         0   0.00    0.00  6,509         0      0   

                                      mrktg                     taxutlt  \
           dsgncst lcnscst  spvsn rentagncy salescsltg advtsmnt prpttax   
2021-12-31       0       0   0.00         0          0        0    0.00   
2022-01-31     624     528  20.42         0      1,040        0    0.00   
2022-02-28       0       0  20.42         0          0        0    0.00   
2022-03-31       0       0  20.42         0          0        0    0.00   
2022-04-30       0       0  20.42         0          0        0    0.00   
2022-05-31       0       0  20.42         0          0        0    0.00   
2022-06-30       0       0  20.42         0          0      250   37.00   
2022-07-31       0       0  20.42         0          0        0    0.00   
2022-08-31       0       0  20.42         0          0        0    0.00   
2022-09-30       0       0  20.42         0          0        0    0.00   
2022-10-31       0       0  20.42         0          0        0    0.00   
2022-11-30       0       0  20.42         0          0        0    0.00   


In [134]:
cost_df.sum()

lnd        prchs          14500.0
           brkrg            130.0
           tax_aqstn        675.0
           nhbond            36.0
           jdclcst           15.0
cstrn      cstrn          65088.0
cstrnidrt  wtrelec          980.0
           wsctbn           163.0
           dsgncst          624.0
           lcnscst          528.0
           spvsn            490.0
mrktg      rentagncy       1070.0
           salescsltg      1040.0
           advtsmnt         500.0
taxutlt    prpttax           74.0
           prsvtntax       2866.0
addtnl     pmfee            200.0
           oprtgcst         900.0
           rsrvfund        1879.0
           trustcst        1879.0
           csltcst          150.0
           legalcst          40.0
fncg       arngmnt         1940.0
           spcoprtg          66.0
           agntbank          60.0
           prepchs_fee        0.0
sum        sum            95893.0
dtype: float64

# Execution Cash Flow

In [135]:
acc = cf.Intlz_accounts(idx, ['oprtg', 'sales', 'repay'])

In [136]:
# Execute cash flow
for idxno in idx.index:
    #### Loans: set loan withdrawble ####
    # If it's initial date then set loan withdrawble.
    equity.set_wtdrbl_intldate(idxno, idx[0])
    loan.tra.set_wtdrbl_intldate(idxno)
    loan.trb.set_wtdrbl_intldate(idxno)
    #loan.trc.set_wtdrbl_intldate(idxno)
    
    #### Cash Inflow: cash inflow from sales or rent etc. ####
    salesamt = sales.sub_scdd[idxno]
    if salesamt > 0:
        sales.send(idxno, salesamt, acc.repay)
    
    #### Expected Costs: calculate expected costs ####
    # calculate operating costs
    oprtg_cost = cost['all'].add_scdd[idxno]

    # calculate financial costs
    for trch, loaneach in loan.dct.items():
        if idxno == loaneach.idxfn[0]:
            loaneach.fee.addscdd(idxno, loaneach.fee.amt)
        if all([loaneach.is_wtdrbl, not loaneach.is_repaid]):
            #tmp_ntnl = -loaneach.ntnl.bal_strt[idxno]
            #tmp_IRamt = tmp_ntnl * loaneach.IR.rate_cycle
            loaneach.IR.addscdd(idxno, loaneach.IRamt_topay(idxno))
    
    # calculate financial fee on balance on Tr.A
    ntnl_sub_rsdl_tra = loan.tra.ntnl_sub_rsdl(idxno)
    feeonbal = ntnl_sub_rsdl_tra * loan.feeonbal_tra / 12
    loan.tra.fee.addscdd(idxno, feeonbal)
    
    # gathering financial costs
    fncl_fee = loan.ttl.fee.add_scdd[idxno]
    fncl_IR = loan.ttl.IR.add_scdd[idxno]

    cost_ttl = oprtg_cost + fncl_fee + fncl_IR
    
    
    #### Loans: withdraw loan ####
    # calculate the amount to withdraw
    amt_rqrd = acc.oprtg.amt_rqrd_excess(idxno, cost_ttl)
    
    # withdraw loan amount
    amt_wtdrw = 0
    amt_wtdrw += equity.wtdrw(idxno, equity.amt_intl, acc.oprtg)
    if idxno == idx.loan[0]:
        #amt_wtdrw += loan.trc.wtdrw(idxno, loan.trc.amt_intl, acc.oprtg)
        amt_wtdrw += loan.trb.wtdrw(idxno, loan.trb.amt_intl, acc.oprtg)
        amt_wtdrw += loan.tra.wtdrw(idxno, loan.tra.amt_intl, acc.oprtg)
    
    amt_rqrd = max(amt_rqrd - amt_wtdrw, 0)
    #amt_rqrd = max(amt_rqrd - loan.trc.wtdrw(idxno, amt_rqrd, acc.oprtg), 0)
    amt_rqrd = max(amt_rqrd - loan.trb.wtdrw(idxno, amt_rqrd, acc.oprtg), 0)
    amt_rqrd = max(amt_rqrd - loan.tra.wtdrw(idxno, amt_rqrd, acc.oprtg), 0)

    
    #### Costs: 토지비, 공사비 등 각종 비용 지출 ####
    for cst_name, cst_acc in cost['all'].dct.items():
        amt_scdd = cst_acc.add_scdd[idxno]
        acc.oprtg.send(idxno, amt_scdd, cst_acc)
        
    
    #### Loans: pay financial cost ####
    acc.oprtg.send(idxno, loan.tra.fee.add_scdd[idxno], loan.tra.fee)
    acc.oprtg.send(idxno, loan.trb.fee.add_scdd[idxno], loan.trb.fee)
    #acc.oprtg.send(idxno, loan.trc.fee.add_scdd[idxno], loan.trc.fee)
    
    acc.oprtg.send(idxno, loan.tra.IR.add_scdd[idxno], loan.tra.IR)
    acc.oprtg.send(idxno, loan.trb.IR.add_scdd[idxno], loan.trb.IR)
    #acc.oprtg.send(idxno, loan.trc.IR.add_scdd[idxno], loan.trc.IR)
    
    
    #### Loans: repay loan amount ####
    if idxno >= loan.idxfn[-1]: # 만기 도래 여부 확인
        amtrpy = loan.tra.repay_amt(idxno, acc.repay.bal_end[idxno])
        acc.repay.send(idxno, amtrpy, loan.tra.ntnl)
        loan.tra.set_repaid(idxno)
                
        if loan.tra.is_repaid:
            amtrpy = loan.trb.repay_amt(idxno, acc.repay.bal_end[idxno])
            acc.repay.send(idxno, amtrpy, loan.trb.ntnl)
            loan.trb.set_repaid(idxno)
        
        if loan.trb.is_repaid:
            acc.repay.send(idxno, acc.repay.bal_end[idxno], acc.oprtg)
        
        #if loan.trb.is_repaid:
        #    amtrpy = loan.trc.repay_amt(idxno, acc.repay.bal_end[idxno])
        #    acc.repay.send(idxno, amtrpy, loan.trc.ntnl)
        #    loan.trc.set_repaid(idxno)
            
        #if loan.trc.is_repaid:
        #    acc.repay.send(idxno, acc.repay.bal_end[idxno], acc.oprtg)
        
    
    #### Loans: Set back loan unwithdrawble at maturity ####
    # If it was maturity date then set back loan unwithdrawble.
    equity.setback_wtdrbl_mtrt(idxno)
    loan.tra.setback_wtdrbl_mtrt(idxno)
    loan.trb.setback_wtdrbl_mtrt(idxno)
    #loan.trc.setback_wtdrbl_mtrt(idxno)

# Analysis of Results

### 1) Accounts

In [137]:
print("## Operating Account ##")
display(R(acc.oprtg.df))

## Operating Account ##


,add_scdd,sub_scdd,bal_strt,amt_add,amt_sub,bal_end
2021-12-31,0.00,0.00,0,"1,300","1,084",216
2022-01-31,0.00,0.00,216,"40,000","25,727","14,489"
2022-02-28,0.00,0.00,"14,489",0,"2,706","11,783"
2022-03-31,0.00,0.00,"11,783",0,"2,706","9,076"
2022-04-30,0.00,0.00,"9,076",0,"2,706","6,370"
2022-05-31,0.00,0.00,"6,370",0,"2,706","3,664"
2022-06-30,0.00,0.00,"3,664",300,"3,933",30
2022-07-31,0.00,0.00,30,"2,700","2,707",23
2022-08-31,0.00,0.00,23,"2,700","2,716",7
2022-09-30,0.00,0.00,7,"2,800","2,724",83


In [138]:
print("## Sales Account ##")
display(R(sales.df))

## Sales Account ##


,add_scdd,sub_scdd,bal_strt,amt_add,amt_sub,bal_end
2021-12-31,0.00,0,0,0,0,0
2022-01-31,0.00,0,0,"130,000",0,"130,000"
2022-02-28,0.00,0,"130,000",0,0,"130,000"
2022-03-31,0.00,0,"130,000",0,0,"130,000"
2022-04-30,0.00,0,"130,000",0,0,"130,000"
2022-05-31,0.00,0,"130,000",0,0,"130,000"
2022-06-30,0.00,0,"130,000",0,0,"130,000"
2022-07-31,0.00,0,"130,000",0,0,"130,000"
2022-08-31,0.00,0,"130,000",0,0,"130,000"
2022-09-30,0.00,0,"130,000",0,0,"130,000"


In [139]:
print("## Repayment Account ##")
display(R(acc.repay.df))

## Repayment Account ##


,add_scdd,sub_scdd,bal_strt,amt_add,amt_sub,bal_end
2021-12-31,0.00,0.00,0.00,0,0,0.00
2022-01-31,0.00,0.00,0.00,0,0,0.00
2022-02-28,0.00,0.00,0.00,0,0,0.00
2022-03-31,0.00,0.00,0.00,0,0,0.00
2022-04-30,0.00,0.00,0.00,0,0,0.00
2022-05-31,0.00,0.00,0.00,0,0,0.00
2022-06-30,0.00,0.00,0.00,0,0,0.00
2022-07-31,0.00,0.00,0.00,0,0,0.00
2022-08-31,0.00,0.00,0.00,0,0,0.00
2022-09-30,0.00,0.00,0.00,0,0,0.00


### 2) Costs

In [140]:
cost_dct = cost["all"].dct.items()
cost_df = DataFrame({key:item.amt_add[:] for key, item in cost_dct})
cost_df[("sum", "sum")] = cost["all"].amt_add[:]
display(R(cost_df))

lnd                                  cstrn cstrnidrt         \
             prchs  brkrg tax_aqstn nhbond jdclcst  cstrn   wtrelec wsctbn   
2021-12-31   1,000  39.00         0   0.00    0.00     35         0      0   
2022-01-31  13,500  91.00       675  36.00   15.00  2,439         0      0   
2022-02-28       0   0.00         0   0.00    0.00  2,439         0      0   
2022-03-31       0   0.00         0   0.00    0.00  2,439         0      0   
2022-04-30       0   0.00         0   0.00    0.00  2,439         0      0   
2022-05-31       0   0.00         0   0.00    0.00  2,439         0      0   
2022-06-30       0   0.00         0   0.00    0.00  2,439         0      0   
2022-07-31       0   0.00         0   0.00    0.00  2,439         0      0   
2022-08-31       0   0.00         0   0.00    0.00  2,439         0      0   
2022-09-30       0   0.00         0   0.00    0.00  2,439         0      0   
2022-10-31       0   0.00         0   0.00    0.00  2,439         0      0   
2022-11-30       0   0.00         0   0.00    0.00  2,439         0      0   
2022-12-31       0   0.00         0   0.00    0.00  2,439         0      0   
2023-01-31       0   0.00         0   0.00    0.00  2,439         0      0   
2023-02-28       0   0.00         0   0.00    0.00  2,439         0      0   
2023-03-31       0   0.00         0   0.00    0.00  2,439         0      0   
2023-04-30       0   0.00         0   0.00    0.00  2,439         0      0   
2023-05-31       0   0.00         0   0.00    0.00  2,439         0      0   
2023-06-30       0   0.00         0   0.00    0.00  2,439         0      0   
2023-07-31       0   0.00         0   0.00    0.00  2,439         0      0   
2023-08-31       0   0.00         0   0.00    0.00  2,439         0      0   
2023-09-30       0   0.00         0   0.00    0.00  2,439         0      0   
2023-10-31       0   0.00         0   0.00    0.00  2,439       980    163   
2023-11-30       0   0.00         0   0.00    0.00  2,439         0      0   
2023-12-31       0   0.00         0   0.00    0.00  2,439         0      0   
2024-01-31       0   0.00         0   0.00    0.00      0         0      0   
2024-02-29       0   0.00         0   0.00    0.00      0         0      0   
2024-03-31       0   0.00         0   0.00    0.00      0         0      0   
2024-04-30       0   0.00         0   0.00    0.00      0         0      0   
2024-05-31       0   0.00         0   0.00    0.00      0         0      0   
2024-06-30       0   0.00         0   0.00    0.00      0         0      0   
2024-07-31       0   0.00         0   0.00    0.00      0         0      0   
2024-08-31       0   0.00         0   0.00    0.00      0         0      0   
2024-09-30       0   0.00         0   0.00    0.00      0         0      0   
2024-10-31       0   0.00         0   0.00    0.00      0         0      0   
2024-11-30       0   0.00         0   0.00    0.00  6,509         0      0   

                                      mrktg                     taxutlt  \
           dsgncst lcnscst  spvsn rentagncy salescsltg advtsmnt prpttax   
2021-12-31       0       0   0.00         0          0        0    0.00   
2022-01-31     624     528  20.42         0      1,040        0    0.00   
2022-02-28       0       0  20.42         0          0        0    0.00   
2022-03-31       0       0  20.42         0          0        0    0.00   
2022-04-30       0       0  20.42         0          0        0    0.00   
2022-05-31       0       0  20.42         0          0        0    0.00   
2022-06-30       0       0  20.42         0          0      250   37.00   
2022-07-31       0       0  20.42         0          0        0    0.00   
2022-08-31       0       0  20.42         0          0        0    0.00   
2022-09-30       0       0  20.42         0          0        0    0.00   
2022-10-31       0       0  20.42         0          0        0    0.00   
2022-11-30       0       0  20.42         0          0        0    0.00   


In [141]:
print("## 필수사업비/선택적사업비 ##")
cost_initial = cost_df[:1].sum()
cost_vital = cost_df[1:-2].sum()
cost_optional = cost_df[-1:].sum()
cost_total = cost_df.sum()
R(DataFrame({'initial':cost_initial,
             'vital':cost_vital,
             'optional':cost_optional,
             'total':cost_total}))

## 필수사업비/선택적사업비 ##


initial   vital optional   total
lnd       prchs         1,000  13,500        0  14,500
          brkrg            39      91        0     130
          tax_aqstn         0     675        0     675
          nhbond            0      36        0      36
          jdclcst           0      15        0      15
cstrn     cstrn            35  58,544    6,509  65,088
cstrnidrt wtrelec           0     980        0     980
          wsctbn            0     163        0     163
          dsgncst           0     624        0     624
          lcnscst           0     528        0     528
          spvsn             0     490        0     490
mrktg     rentagncy         0       0    1,070   1,070
          salescsltg        0   1,040        0   1,040
          advtsmnt          0     500        0     500
taxutlt   prpttax           0      74        0      74
          prsvtntax         0   2,866        0   2,866
addtnl    pmfee             0     200        0     200
          oprtgcst          0     900        0     900
          rsrvfund          0   1,879        0   1,879
          trustcst          0   1,879        0   1,879
          csltcst          10     140        0     150
          legalcst          0      40        0      40
fncg      arngmnt           0   1,940        0   1,940
          spcoprtg          0      66        0      66
          agntbank          0      60        0      60
          prepchs_fee       0       0        0       0
sum       sum           1,084  87,230    7,579  95,893

### 3) Loan

#### 3-1) Notional Amount

In [142]:
loan_dct = loan.ttl.dct.items()

loan_df = {}
for key, item in loan_dct:
    loan_df[(key, 'amt_sub')] = item.ntnl.amt_sub[:]
    loan_df[(key, 'amt_add')] = item.ntnl.amt_add[:]
    loan_df[(key, 'bal_end')] = item.ntnl.bal_end[:]
loan_df = DataFrame(loan_df)
loan_df[('sum', 'amt_sub')] = sum([item for (key1, key2), item in loan_df.items() if key2 == "amt_sub"])
loan_df[('sum', 'amt_add')] = sum([item for (key1, key2), item in loan_df.items() if key2 == "amt_add"])
loan_df[('sum', 'bal_end')] = sum([item for (key1, key2), item in loan_df.items() if key2 == "bal_end"])

display(R(loan_df))

tra                      trb                      sum          \
           amt_sub amt_add  bal_end amt_sub amt_add  bal_end amt_sub amt_add   
2021-12-31       0       0        0       0       0        0       0       0   
2022-01-31  18,000       0  -18,000  22,000       0  -22,000  40,000       0   
2022-02-28       0       0  -18,000       0       0  -22,000       0       0   
2022-03-31       0       0  -18,000       0       0  -22,000       0       0   
2022-04-30       0       0  -18,000       0       0  -22,000       0       0   
2022-05-31       0       0  -18,000       0       0  -22,000       0       0   
2022-06-30     300       0  -18,300       0       0  -22,000     300       0   
2022-07-31   2,700       0  -21,000       0       0  -22,000   2,700       0   
2022-08-31   2,700       0  -23,700       0       0  -22,000   2,700       0   
2022-09-30   2,800       0  -26,500       0       0  -22,000   2,800       0   
2022-10-31   2,700       0  -29,200       0       0  -22,000   2,700       0   
2022-11-30   2,700       0  -31,900       0       0  -22,000   2,700       0   
2022-12-31   2,800       0  -34,700       0       0  -22,000   2,800       0   
2023-01-31   2,800       0  -37,500       0       0  -22,000   2,800       0   
2023-02-28   2,700       0  -40,200       0       0  -22,000   2,700       0   
2023-03-31   2,800       0  -43,000       0       0  -22,000   2,800       0   
2023-04-30   2,800       0  -45,800       0       0  -22,000   2,800       0   
2023-05-31   2,800       0  -48,600       0       0  -22,000   2,800       0   
2023-06-30   4,000       0  -52,600       0       0  -22,000   4,000       0   
2023-07-31   2,900       0  -55,500       0       0  -22,000   2,900       0   
2023-08-31   2,800       0  -58,300       0       0  -22,000   2,800       0   
2023-09-30   2,800       0  -61,100       0       0  -22,000   2,800       0   
2023-10-31   4,000       0  -65,100       0       0  -22,000   4,000       0   
2023-11-30   2,900       0  -68,000       0       0  -22,000   2,900       0   
2023-12-31   5,700       0  -73,700       0       0  -22,000   5,700       0   
2024-01-31     400       0  -74,100       0       0  -22,000     400       0   
2024-02-29     400       0  -74,500       0       0  -22,000     400       0   
2024-03-31     500       0  -75,000       0       0  -22,000     500       0   
2024-04-30       0       0  -75,000       0       0  -22,000       0       0   
2024-05-31       0       0  -75,000       0       0  -22,000       0       0   
2024-06-30       0  75,000        0       0  22,000        0       0  97,000   
2024-07-31       0       0        0       0       0        0       0       0   
2024-08-31       0       0        0       0       0        0       0       0   
2024-09-30       0       0        0       0       0        0       0       0   
2024-10-31       0       0        0       0       0        0       0       0   
2024-11-30       0       0        0       0       0        0       0       0   

                     
            bal_end  
2021-12-31        0  
2022-01-31  -40,000  
2022-02-28  -40,000  
2022-03-31  -40,000  
2022-04-30  -40,000  
2022-05-31  -40,000  
2022-06-30  -40,300  
2022-07-31  -43,000  
2022-08-31  -45,700  
2022-09-30  -48,500  
2022-10-31  -51,200  
2022-11-30  -53,900  
2022-12-31  -56,700  
2023-01-31  -59,500  
2023-02-28  -62,200  
2023-03-31  -65,000  
2023-04-30  -67,800  
2023-05-31  -70,600  
2023-06-30  -74,600  
2023-07-31  -77,500  
2023-08-31  -80,300  
2023-09-30  -83,100  
2023-10-31  -87,100  
2023-11-30  -90,000  
2023-12-31  -95,700  
2024-01-31  -96,100  
2024-02-29  -96,500  
2024-03-31  -97,000  
2024-04-30  -97,000  
2024-05-31  -97,000  
2024-06-30        0  
2024-07-31        0  
2024-08-31        0  
2024-09-30        0  
2024-10-31        0  
2024-11-30        0

#### 3-2) Fee Amount

In [143]:
fee_dct = DataFrame({key: sum(item.amt_add[:]) for key, item in loan.ttl.fee.dct.items()}, index=['amt_add'])
fee_dct['sum'] = fee_dct.sum(axis=1)
R(fee_dct)

,tra,trb,sum
amt_add,"1,897",902,"2,799"


In [144]:
loan_dct = loan.ttl.dct.items()

loan_df = {}
for key, item in loan_dct:
    loan_df[(key, 'amt_sub')] = item.fee.amt_sub[:]
    loan_df[(key, 'amt_add')] = item.fee.amt_add[:]
    loan_df[(key, 'bal_end')] = item.fee.bal_end[:]
loan_df = DataFrame(loan_df)
loan_df[('sum', 'amt_sub')] = sum([item for (key1, key2), item in loan_df.items() if key2 == "amt_sub"])
loan_df[('sum', 'amt_add')] = sum([item for (key1, key2), item in loan_df.items() if key2 == "amt_add"])
loan_df[('sum', 'bal_end')] = sum([item for (key1, key2), item in loan_df.items() if key2 == "bal_end"])

display(R(loan_df))

tra                     trb                     sum          \
           amt_sub amt_add bal_end amt_sub amt_add bal_end amt_sub amt_add   
2021-12-31    0.00       0       0    0.00       0       0    0.00       0   
2022-01-31    0.00   1,531   1,531    0.00     902     902    0.00   2,433   
2022-02-28    0.00      24   1,555    0.00       0     902    0.00      24   
2022-03-31    0.00      24   1,579    0.00       0     902    0.00      24   
2022-04-30    0.00      24   1,602    0.00       0     902    0.00      24   
2022-05-31    0.00      24   1,626    0.00       0     902    0.00      24   
2022-06-30    0.00      24   1,650    0.00       0     902    0.00      24   
2022-07-31    0.00      24   1,674    0.00       0     902    0.00      24   
2022-08-31    0.00      22   1,696    0.00       0     902    0.00      22   
2022-09-30    0.00      21   1,718    0.00       0     902    0.00      21   
2022-10-31    0.00      20   1,738    0.00       0     902    0.00      20   
2022-11-30    0.00      19   1,757    0.00       0     902    0.00      19   
2022-12-31    0.00      18   1,775    0.00       0     902    0.00      18   
2023-01-31    0.00      17   1,792    0.00       0     902    0.00      17   
2023-02-28    0.00      16   1,807    0.00       0     902    0.00      16   
2023-03-31    0.00      14   1,822    0.00       0     902    0.00      14   
2023-04-30    0.00      13   1,835    0.00       0     902    0.00      13   
2023-05-31    0.00      12   1,847    0.00       0     902    0.00      12   
2023-06-30    0.00      11   1,858    0.00       0     902    0.00      11   
2023-07-31    0.00       9   1,867    0.00       0     902    0.00       9   
2023-08-31    0.00       8   1,876    0.00       0     902    0.00       8   
2023-09-30    0.00       7   1,883    0.00       0     902    0.00       7   
2023-10-31    0.00       6   1,888    0.00       0     902    0.00       6   
2023-11-30    0.00       4   1,892    0.00       0     902    0.00       4   
2023-12-31    0.00       3   1,895    0.00       0     902    0.00       3   
2024-01-31    0.00       1   1,896    0.00       0     902    0.00       1   
2024-02-29    0.00       0   1,896    0.00       0     902    0.00       0   
2024-03-31    0.00       0   1,897    0.00       0     902    0.00       0   
2024-04-30    0.00       0   1,897    0.00       0     902    0.00       0   
2024-05-31    0.00       0   1,897    0.00       0     902    0.00       0   
2024-06-30    0.00       0   1,897    0.00       0     902    0.00       0   
2024-07-31    0.00       0   1,897    0.00       0     902    0.00       0   
2024-08-31    0.00       0   1,897    0.00       0     902    0.00       0   
2024-09-30    0.00       0   1,897    0.00       0     902    0.00       0   
2024-10-31    0.00       0   1,897    0.00       0     902    0.00       0   
2024-11-30    0.00       0   1,897    0.00       0     902    0.00       0   

                    
           bal_end  
2021-12-31       0  
2022-01-31   2,433  
2022-02-28   2,457  
2022-03-31   2,481  
2022-04-30   2,504  
2022-05-31   2,528  
2022-06-30   2,552  
2022-07-31   2,576  
2022-08-31   2,598  
2022-09-30   2,620  
2022-10-31   2,640  
2022-11-30   2,659  
2022-12-31   2,677  
2023-01-31   2,694  
2023-02-28   2,709  
2023-03-31   2,724  
2023-04-30   2,737  
2023-05-31   2,749  
2023-06-30   2,760  
2023-07-31   2,770  
2023-08-31   2,778  
2023-09-30   2,785  
2023-10-31   2,790  
2023-11-30   2,794  
2023-12-31   2,797  
2024-01-31   2,798  
2024-02-29   2,798  
2024-03-31   2,799  
2024-04-30   2,799  
2024-05-31   2,799  
2024-06-30   2,799  
2024-07-31   2,799  
2024-08-31   2,799  
2024-09-30   2,799  
2024-10-31   2,799  
2024-11-30   2,799

#### 3-3) IR Amount

In [145]:
IR_dct = DataFrame({key: sum(item.amt_add[:]) for key, item in loan.ttl.IR.dct.items()}, index=['amt_add'])
IR_dct['sum'] = IR_dct.sum(axis=1)
R(IR_dct)

,tra,trb,sum
amt_add,"4,652","3,722","8,374"


In [146]:
loan_dct = loan.ttl.dct.items()

loan_df = {}
for key, item in loan_dct:
    loan_df[(key, 'amt_sub')] = item.IR.amt_sub[:]
    loan_df[(key, 'amt_add')] = item.IR.amt_add[:]
    loan_df[(key, 'bal_end')] = item.IR.bal_end[:]
loan_df = DataFrame(loan_df)
loan_df[('sum', 'amt_sub')] = sum([item for (key1, key2), item in loan_df.items() if key2 == "amt_sub"])
loan_df[('sum', 'amt_add')] = sum([item for (key1, key2), item in loan_df.items() if key2 == "amt_add"])
loan_df[('sum', 'bal_end')] = sum([item for (key1, key2), item in loan_df.items() if key2 == "bal_end"])

display(R(loan_df))

tra                     trb                     sum          \
           amt_sub amt_add bal_end amt_sub amt_add bal_end amt_sub amt_add   
2021-12-31    0.00       0       0    0.00       0       0    0.00       0   
2022-01-31    0.00       0       0    0.00       0       0    0.00       0   
2022-02-28    0.00      64      64    0.00     128     128    0.00     193   
2022-03-31    0.00      64     129    0.00     128     257    0.00     193   
2022-04-30    0.00      64     193    0.00     128     385    0.00     193   
2022-05-31    0.00      64     258    0.00     128     513    0.00     193   
2022-06-30    0.00      64     322    0.00     128     642    0.00     193   
2022-07-31    0.00      66     388    0.00     128     770    0.00     194   
2022-08-31    0.00      75     463    0.00     128     898    0.00     204   
2022-09-30    0.00      85     548    0.00     128   1,027    0.00     213   
2022-10-31    0.00      95     643    0.00     128   1,155    0.00     223   
2022-11-30    0.00     105     748    0.00     128   1,283    0.00     233   
2022-12-31    0.00     114     862    0.00     128   1,412    0.00     243   
2023-01-31    0.00     124     986    0.00     128   1,540    0.00     253   
2023-02-28    0.00     134   1,121    0.00     128   1,668    0.00     263   
2023-03-31    0.00     144   1,265    0.00     128   1,797    0.00     272   
2023-04-30    0.00     154   1,419    0.00     128   1,925    0.00     282   
2023-05-31    0.00     164   1,583    0.00     128   2,053    0.00     292   
2023-06-30    0.00     174   1,757    0.00     128   2,182    0.00     302   
2023-07-31    0.00     188   1,946    0.00     128   2,310    0.00     317   
2023-08-31    0.00     199   2,145    0.00     128   2,438    0.00     327   
2023-09-30    0.00     209   2,354    0.00     128   2,567    0.00     337   
2023-10-31    0.00     219   2,572    0.00     128   2,695    0.00     347   
2023-11-30    0.00     233   2,806    0.00     128   2,823    0.00     362   
2023-12-31    0.00     244   3,049    0.00     128   2,952    0.00     372   
2024-01-31    0.00     264   3,314    0.00     128   3,080    0.00     392   
2024-02-29    0.00     266   3,579    0.00     128   3,208    0.00     394   
2024-03-31    0.00     267   3,846    0.00     128   3,337    0.00     395   
2024-04-30    0.00     269   4,115    0.00     128   3,465    0.00     397   
2024-05-31    0.00     269   4,383    0.00     128   3,593    0.00     397   
2024-06-30    0.00     269   4,652    0.00     128   3,722    0.00     397   
2024-07-31    0.00       0   4,652    0.00       0   3,722    0.00       0   
2024-08-31    0.00       0   4,652    0.00       0   3,722    0.00       0   
2024-09-30    0.00       0   4,652    0.00       0   3,722    0.00       0   
2024-10-31    0.00       0   4,652    0.00       0   3,722    0.00       0   
2024-11-30    0.00       0   4,652    0.00       0   3,722    0.00       0   

                    
           bal_end  
2021-12-31       0  
2022-01-31       0  
2022-02-28     193  
2022-03-31     386  
2022-04-30     578  
2022-05-31     771  
2022-06-30     964  
2022-07-31   1,158  
2022-08-31   1,362  
2022-09-30   1,575  
2022-10-31   1,798  
2022-11-30   2,031  
2022-12-31   2,274  
2023-01-31   2,526  
2023-02-28   2,789  
2023-03-31   3,062  
2023-04-30   3,344  
2023-05-31   3,636  
2023-06-30   3,939  
2023-07-31   4,256  
2023-08-31   4,583  
2023-09-30   4,920  
2023-10-31   5,267  
2023-11-30   5,629  
2023-12-31   6,001  
2024-01-31   6,394  
2024-02-29   6,787  
2024-03-31   7,183  
2024-04-30   7,580  
2024-05-31   7,977  
2024-06-30   8,374  
2024-07-31   8,374  
2024-08-31   8,374  
2024-09-30   8,374  
2024-10-31   8,374  
2024-11-30   8,374